# This file reads stream from kafka producer and proccess it then save it as silver layer in adls gen2 
It should be running all the time

In [0]:
# Configuration Variables
eh_namespace = "xxx"
eh_name = "xxx"
# Best practice: Retrieve from Databricks Secrets
eh_connection_string = "Endpoint=sb://xxx.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=xxxx"

# Kafka Connection Options
kafka_options = {
    "kafka.bootstrap.servers": f"{eh_namespace}.servicebus.windows.net:9093",
    "subscribe": eh_name,
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{eh_connection_string}";',
    "startingOffsets": "latest",
    "failOnDataLoss": "false"
}

# Read the Stream
df = (spark.readStream
      .format("kafka")
      .options(**kafka_options)
      .load())

# The 'value' column is binary by default; cast it to string to read JSON/text
stream_df = df.selectExpr("CAST(value AS STRING)")


In [0]:
# display(stream_df)

value
"{""customer_id"": ""8870-HNKHH"", ""gender"": ""Female"", ""senior_citizen"": true, ""partner"": true, ""dependents"": false, ""tenure"": 62, ""phone_service"": true, ""multiple_lines"": ""Yes"", ""internet_service"": ""DSL"", ""online_security"": ""No internet service"", ""online_backup"": ""Yes"", ""device_protection"": ""No"", ""tech_support"": ""No internet service"", ""streaming_tv"": ""No"", ""streaming_movies"": ""No"", ""contract"": ""Two year"", ""paperless_billing"": true, ""payment_method"": ""Bank transfer"", ""monthly_charges"": 134.28, ""total_charges"": 8466.3, ""updated_at"": ""2026-01-27T08:26:32.850185Z"", ""first_name"": ""Michelle"", ""last_name"": ""Walker"", ""email"": ""cgood@example.org""}"
"{""customer_id"": ""0276-UHWWL"", ""gender"": ""Female"", ""senior_citizen"": false, ""partner"": true, ""dependents"": true, ""tenure"": 38, ""phone_service"": true, ""multiple_lines"": ""Yes"", ""internet_service"": ""Fiber optic"", ""online_security"": ""No"", ""online_backup"": ""Yes"", ""device_protection"": ""No"", ""tech_support"": ""No"", ""streaming_tv"": ""No"", ""streaming_movies"": ""No"", ""contract"": ""One year"", ""paperless_billing"": true, ""payment_method"": ""Bank transfer"", ""monthly_charges"": 66.28, ""total_charges"": 2415.54, ""updated_at"": ""2026-01-27T08:26:37.853867Z"", ""first_name"": ""Hector"", ""last_name"": ""Rodriguez"", ""email"": ""wthompson@example.net""}"
"{""customer_id"": ""8842-CCCDT"", ""gender"": ""Female"", ""senior_citizen"": true, ""partner"": true, ""dependents"": false, ""tenure"": 67, ""phone_service"": true, ""multiple_lines"": ""No"", ""internet_service"": ""None"", ""online_security"": ""No internet service"", ""online_backup"": ""No"", ""device_protection"": ""Yes"", ""tech_support"": ""Yes"", ""streaming_tv"": ""No"", ""streaming_movies"": ""Yes"", ""contract"": ""Month-to-month"", ""paperless_billing"": true, ""payment_method"": ""Mailed check"", ""monthly_charges"": 75.33, ""total_charges"": 5395.02, ""updated_at"": ""2026-01-27T08:26:42.855953Z"", ""first_name"": ""Dale"", ""last_name"": ""Diaz"", ""email"": ""hernandezmichael@example.net""}"
"{""customer_id"": ""9232-XCGIH"", ""gender"": ""Female"", ""senior_citizen"": false, ""partner"": false, ""dependents"": true, ""tenure"": 60, ""phone_service"": false, ""multiple_lines"": ""Yes"", ""internet_service"": ""None"", ""online_security"": ""Yes"", ""online_backup"": ""No internet service"", ""device_protection"": ""Yes"", ""tech_support"": ""No"", ""streaming_tv"": ""Yes"", ""streaming_movies"": ""No"", ""contract"": ""Month-to-month"", ""paperless_billing"": true, ""payment_method"": ""Electronic check"", ""monthly_charges"": 23.51, ""total_charges"": 1375.51, ""updated_at"": ""2026-01-27T08:26:47.857039Z"", ""first_name"": ""Kimberly"", ""last_name"": ""Franklin"", ""email"": ""autumnmyers@example.com""}"
"{""customer_id"": ""9643-OWRLH"", ""gender"": ""Male"", ""senior_citizen"": false, ""partner"": true, ""dependents"": false, ""tenure"": 30, ""phone_service"": true, ""multiple_lines"": ""No"", ""internet_service"": ""DSL"", ""online_security"": ""No"", ""online_backup"": ""No internet service"", ""device_protection"": ""No"", ""tech_support"": ""No"", ""streaming_tv"": ""No"", ""streaming_movies"": ""Yes"", ""contract"": ""Two year"", ""paperless_billing"": true, ""payment_method"": ""Credit card"", ""monthly_charges"": 128.21, ""total_charges"": 3766.99, ""updated_at"": ""2026-01-27T08:26:52.859763Z"", ""first_name"": ""Cory"", ""last_name"": ""Vasquez"", ""email"": ""vobrien@example.net""}"
"{""customer_id"": ""0376-UMIZV"", ""gender"": ""Male"", ""senior_citizen"": false, ""partner"": false, ""dependents"": false, ""tenure"": 52, ""phone_service"": true, ""multiple_lines"": ""No"", ""internet_service"": ""DSL"", ""online_security"": ""No"", ""online_backup"": ""No internet service"", ""device_protection"": ""No"", ""tech_support"": ""No"", ""streaming_tv"": ""No"", ""streaming_movies"": ""No""

PROCESSING ----------------------------------

In [0]:
from pyspark.sql.types import *

customer_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("senior_citizen", BooleanType(), True),
    StructField("partner", BooleanType(), True),
    StructField("dependents", BooleanType(), True),
    StructField("tenure", IntegerType(), True),
    StructField("phone_service", BooleanType(), True),
    StructField("multiple_lines", StringType(), True),
    StructField("internet_service", StringType(), True),
    StructField("online_security", StringType(), True),
    StructField("online_backup", StringType(), True),
    StructField("device_protection", StringType(), True),
    StructField("tech_support", StringType(), True),
    StructField("streaming_tv", StringType(), True),
    StructField("streaming_movies", StringType(), True),
    StructField("contract", StringType(), True),
    StructField("paperless_billing", BooleanType(), True),
    StructField("payment_method", StringType(), True),
    StructField("monthly_charges", DoubleType(), True),
    StructField("total_charges", DoubleType(), True),
    StructField("updated_at", TimestampType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True)
])

In [0]:
from pyspark.sql.functions import from_json, col, when

# Parse JSON into structured DataFrame
parsed_df = stream_df.select(from_json(col("value"), customer_schema).alias("data")).select("data.*")
# parsed_df = stream_df.withColumn("data", from_json(col("value"), customer_schema)).select("data.*")

final_df = (
    parsed_df
    .withColumn("total_charges", 
        when(col("total_charges").isNull(), col("monthly_charges") * col("tenure"))
        .otherwise(col("total_charges"))
    )
    .fillna({
        "multiple_lines": "No",
        "internet_service": "None",
        "online_security": "No",
        "online_backup": "No",
        "device_protection": "No",
        "tech_support": "No",
        "streaming_tv": "No",
        "streaming_movies": "No",
        "contract": "Month-to-month",
        "payment_method": "Electronic check"
    })
    .dropDuplicates(["customer_id"])
)



In [0]:
display(final_df)

customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,updated_at,first_name,last_name,email
2019-MGNKA,Female,false,true,true,2,true,Yes,Fiber optic,No internet service,No,No,No,No,No,Two year,false,Mailed check,43.78,84.37,2026-01-27T08:28:52.90478Z,Joan,Kelly,michellebishop@example.net
6071-GGSMA,Male,false,true,true,45,true,No,DSL,No internet service,No,No,Yes,Yes,No,Month-to-month,true,Bank transfer,50.4,2228.13,2026-01-27T08:28:57.909289Z,Rhonda,Thompson,nicholsjohn@example.com
1346-TMRBT,Male,false,true,false,54,true,Yes,Fiber optic,No,No,No internet service,No internet service,Yes,No internet service,Month-to-month,true,Electronic check,93.76,5429.24,2026-01-27T08:29:02.911778Z,Carlos,Hill,lwalls@example.net
5237-DHKMS,Female,false,false,true,68,true,Yes,None,No,No,No,Yes,No,No,One year,true,Bank transfer,101.09,7064.13,2026-01-27T08:29:07.912593Z,Susan,Griffin,weaverbrenda@example.org
5939-DOCTX,Male,false,true,false,36,true,No,DSL,No,No,Yes,No internet service,Yes,Yes,Month-to-month,false,Electronic check,54.29,2132.4,2026-01-27T08:29:12.914029Z,Audrey,Fletcher,lori30@example.org
6500-MGVGV,Female,false,true,true,16,false,Yes,DSL,No,No,No,No,No,No,One year,true,Electronic check,24.49,418.93,2026-01-27T08:29:17.915274Z,Willie,May,friedmankatie@example.net
9630-DZZFQ,Female,false,true,false,16,true,No,Fiber optic,No,No,No,No,Yes,No,Month-to-month,true,Electronic check,84.8,1294.82,2026-01-27T08:29:27.918118Z,Ashley,Lopez,wmiller@example.org
2353-MGHPX,Male,true,true,false,39,true,Yes,Fiber optic,No internet service,No,No,Yes,No internet service,Yes,One year,false,Electronic check,77.46,2847.59,2026-01-27T08:29:22.917034Z,Kaitlyn,Daniels,johnny76@example.org
0085-JPSGN,Male,false,false,true,19,true,No,DSL,No,No,No,No,No,No,One year,true,Mailed check,83.41,1630.84,2026-01-27T08:29:32.921003Z,Kyle,Diaz,llewis@example.org
0092-WZKGR,Female,false,true,false,7,true,No,Fiber optic,Yes,No,No,No,No,No,Month-to-month,true,Electronic check,140.41,957.28,2026-01-27T08:29:37.922729Z,Alicia,Jones,elizabeth24@example.net


ADLS GEN2-------------------------------------------------

In [0]:
# ======= ADLS Gen2 Config =======
storage_account = "xxx"
container = "xxx" 
client_id = "xxx"
tenant_id = "xxx"
client_secret = "xxx"
# ======= Spark Configuration for OAuth =======
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
)
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(
    f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
    f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
)



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

customer_schema = StructType([
    StructField("customer_id", StringType(), False),
    StructField("gender", StringType(), True),
    StructField("senior_citizen", BooleanType(), True),
    StructField("partner", BooleanType(), True),
    StructField("dependents", BooleanType(), True),
    StructField("tenure", IntegerType(), True),

    StructField("phone_service", BooleanType(), True),
    StructField("multiple_lines", StringType(), True),

    StructField("internet_service", StringType(), True),
    StructField("online_security", StringType(), True),
    StructField("online_backup", StringType(), True),
    StructField("device_protection", StringType(), True),
    StructField("tech_support", StringType(), True),
    StructField("streaming_tv", StringType(), True),
    StructField("streaming_movies", StringType(), True),

    StructField("contract", StringType(), True),
    StructField("paperless_billing", BooleanType(), True),
    StructField("payment_method", StringType(), True),

    StructField("monthly_charges", DecimalType(10, 2), True),
    StructField("total_charges", DecimalType(10, 2), True),

    StructField("updated_at", TimestampType(), False),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True)
])

parsed_df = (
    stream_df
    .select(from_json(col("value"), customer_schema).alias("data"))
    .select("data.*")
)

silver_df = (
    parsed_df
    .withColumn(
        "total_charges",
        when(col("total_charges").isNull(),
             col("monthly_charges") * col("tenure"))
        .otherwise(col("total_charges"))
    )
    .fillna({
        "multiple_lines": "No",
        "internet_service": "None",
        "online_security": "No",
        "online_backup": "No",
        "device_protection": "No",
        "tech_support": "No",
        "streaming_tv": "No",
        "streaming_movies": "No",
        "contract": "Month-to-month",
        "payment_method": "Electronic check"
    })
    .withColumn("event_time", col("updated_at"))
)

silver_df = silver_df.withWatermark("event_time", "10 minutes")

final_stream_df = (
    silver_df
    .withColumn("year", year("event_time"))
    .withColumn("month", month("event_time"))
    .withColumn("day", dayofmonth("event_time"))
    .withColumn("hour", hour("event_time"))
)

In [0]:
adls_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net"

target_path = f"{adls_base_path}/silver/customers"
checkpoint_path = f"{adls_base_path}/checkpoints/customers_stream"

query = (
    final_stream_df.writeStream
    .format("delta")
    .outputMode("append")
    .partitionBy("year", "month", "day", "hour")
    .option("checkpointLocation", checkpoint_path)
    .trigger(processingTime="1 minute")
    .start(target_path)
)
